In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from math import log10, sqrt


In [ ]:
#Load images

snrImg = []
snrReconstruct = []

rootNoisy = "gr5/Noisy"
noisyImgs = glob.glob(f"{rootNoisy}/*.bmp") + glob.glob(f"{rootNoisy}/*.jpg") 

rootOriginal = "gr5/Original"
originalImgs = glob.glob(f"{rootOriginal}/*.bmp") + glob.glob(f"{rootOriginal}/*.jpg")

originalImgs.sort()
noisyImgs.sort()

In [ ]:
#presa un immagine definisco i tipi di rumore

img = cv2.imread(originalImgs[1],0)
np.random.seed(42)

#Rumore gaussiano
noise = np.random.normal(size=img.shape)
imgNoisyGauss = img + noise
imgNoisyGauss = cv2.normalize(imgNoisyGauss, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

histOrig, bins = np.histogram(img.ravel(), bins=256, range=(0, 255))

histNoise, bins = np.histogram(imgNoisyGauss.ravel(), bins=256, range=(0, 255))

histNoiseOnlyGauss = histNoise - histOrig

#Rumore uniforme
low = 0
high = 255
noise = np.random.uniform(low, high, img.shape)
#noise = np.random.rand(*img.shape) * 255
imgNoisyUni = img + noise
imgNoisyUni = cv2.normalize(imgNoisyUni, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

histNoise, bins = np.histogram(imgNoisyUni.ravel(), bins=256, range=(0, 255))

histNoiseOnlyUnif = histNoise - histOrig

#Rumore sale e pepe
x,y = img.shape
noise = np.random.rand(x,y)*255
salt = noise > 240
pepper = noise < 15
imgNoisySandP = img.copy()
imgNoisySandP[salt] = 255
imgNoisySandP[pepper] = 0
imgNoisySandP = cv2.normalize(imgNoisySandP, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

histNoise, bins = np.histogram(imgNoisySandP.ravel(), bins=256, range=(0, 255))

histNoiseOnlySandP = histNoise - histOrig

#matplot delle immagini con istogrammi
fig, axes = plt.subplots(4,2,figsize=(7,7))
fig.tight_layout()
axes[0][0].imshow(img, cmap="gray")
axes[0][0].set_title(f"Immagine originale")

axes[0][1].hist(img.ravel(),256,[0,256])
axes[0][1].set_title(f"Istogramma immagine originale")

axes[1][0].imshow(imgNoisyGauss, cmap="gray")
axes[1][0].set_title(f"Immagine rumorosa gauss")

axes[1][1].hist(histNoiseOnlyGauss,256,[0,256])
axes[1][1].set_title(f"Istogramma rumore gauss")

axes[2][0].imshow(imgNoisyUni, cmap="gray")
axes[2][0].set_title(f"Immagine rumorosa uniforme ")

axes[2][1].hist(histNoiseOnlyUnif,256,[0,256])
axes[2][1].set_title(f"Istogramma rumore uniforme ")

axes[3][0].imshow(imgNoisySandP, cmap="gray")
axes[3][0].set_title(f"Immagine rumorosa sale e pepe ")

axes[3][1].hist(histNoiseOnlySandP,256,[0,256])
axes[3][1].set_title(f"Istogramma rumore sale e pepe ")

plt.show()

In [89]:
def denoiseImage(X, iter=10, WEd=1, WEp=1, exp=1):
  ''' 
  input: X np array for an image, iter int for number of iteration, WEd weight for data energy, WEp weight for data penalty

  output: Denoised imaged
  '''
  m, n = np.shape(X)
  Y = np.copy(X).astype(dtype=float)
  Ep = np.zeros_like(X)
  intesities = np.arange(256)

  for iteration in range(iter):
  
    print(f'Iteration in progress: {iteration+1}/{iter}               ', end="\r")
    
    #Fare quattro matrici (una per pixel in cui si va a finire)
    #poi si somma il pixel interessato moltiplicato per il coefficente
    for i in range(1,m-1):
      for j in range (1,n-1):
        Ep[i][j] = ( np.power(abs(Y[i-1][j] - Y[i][j]),exp) +  np.power(abs(Y[i][j-1] - Y[i][j]),exp ) +  np.power(abs(Y[i+1][j]- Y[i][j]),exp ) +  np.power(abs(Y[i][j+1]- Y[i][j]) , exp ) ) *WEp
    
    Ed = np.copy(X).astype(dtype=float)
    Ed = np.abs(Y - X)*WEd
    EpEd = np.add(Ep, Ed)
      
    for i in range(1,m-1):
      for j in range (1,n-1):
        Epl = np.zeros_like(intesities)
        Edl = np.zeros_like(intesities)
        #abs distanza elevato ad un numero minore di 1
        Epl = ( np.power(abs(intesities - Y[i-1,j]),exp) + np.power(abs(intesities - Y[i,j-1]),exp) + np.power(abs(intesities - Y[i+1,j]),exp) +np.power(abs(intesities - Y[i,j+1]),exp) ) *WEp
        #norma euclidea
        Edl = np.abs(intesities - X[i,j])*WEd

        energy = Epl + Edl
        minEnergy = np.min(energy)
        minIndex = np.argmin(energy)
        if minEnergy < EpEd[i,j]:
              Y[i,j] = (intesities[minIndex])
              EpEd[i,j] = minEnergy
        '''
        high = 255
        low = 0
        while low <= high:
          mid = (high + low) // 2
          Epl = float(((Y[i-1,j]- mid)**2) + ((Y[i,j-1]- mid)**2)+((Y[i+1,j]- mid)**2)+ ((Y[i,j+1]- mid)**2)) * WEp
          Edl = float( (X[i,j] - mid))*WEd
          energy = Epl + Edl
          if energy < EpEd[i,j]:
              Y[i,j] = mid
              EpEd[i,j] = energy
          if Epl < Ep[i,j]:
            high = mid - 1
          else:
            low = mid + 1
          '''

  return Y.astype(dtype=int)


In [111]:
reconstructedImgs = []
#for i in range(len(originalImgs)):
for i in range(1):
    print(f"image#:{i}                    ")
    imgOriginal = cv2.imread(originalImgs[i],0)
    imgNoisy = cv2.imread(noisyImgs[i],0)
    reconstructedImgs.append(denoiseImage(imgNoisy, exp = 0.9 ,iter = 10, WEd=0.05, WEp=0.6))
'''
fig, axes = plt.subplots(3,1,figsize=(10,10))
fig.tight_layout()
axes[0].imshow(imgOriginal,cmap="gray")
axes[0].set_title('Original Image')
axes[1].imshow(imgNoisy,cmap="gray")
axes[1].set_title('Noisy Image')
axes[2].imshow(imgReconstructed, cmap="gray")
axes[2].set_title('Denoised Image')
plt.show()
'''

image#:0                    


'\nfig, axes = plt.subplots(3,1,figsize=(10,10))\nfig.tight_layout()\naxes[0].imshow(imgOriginal,cmap="gray")\naxes[0].set_title(\'Original Image\')\naxes[1].imshow(imgNoisy,cmap="gray")\naxes[1].set_title(\'Noisy Image\')\naxes[2].imshow(imgReconstructed, cmap="gray")\naxes[2].set_title(\'Denoised Image\')\nplt.show()\n'

In [113]:

def PSNR(x1, x2):
    mse = np.mean((x1 - x2) ** 2)
    if(mse == 0):  
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

for i in range(len(reconstructedImgs)):
    imgOriginal = cv2.imread(originalImgs[i],0)
    imgNoisy = cv2.imread(noisyImgs[i],0)
    PSNROriginalNoisy = PSNR(imgOriginal, imgNoisy)
    PSNROriginalyDenoised = PSNR(imgOriginal, reconstructedImgs[i])
    print(f"Image#:{i} \n    PSNR from original to noisy: {round(PSNROriginalNoisy,3)} dB\n    PSNR from original to denoised: {round(PSNROriginalyDenoised,3)} dB")



Image#:0 
    PSNR from original to noisy: 28.08 dB
    PSNR from original to denoised: 23.287 dB


In [92]:
for i in range(len(reconstructedImgs)):

    cv2.imwrite(f"denoisedImages/DenoisedImage#{i}.jpeg",reconstructedImgs[i])
